In [8]:
import torch
import math
torch.set_grad_enabled(False)

In [ ]:
#build networks combining fully connected layers, Tanh, and ReLU,
#run the forward and backward passes,
#optimize parameters with SGD for MSE.

# LINEAR

In [41]:
#LINEAR MODULE (FULLY CONNECTED LAYER)
class Linear(object):
    def __init__(self):
        k_root = math.sqrt(1/in_feature)
        self.weight = torch.empty(in_feature,out_feature).uniform_(-k_p,k_p)
        self.bias = torch.empty(out_feature).uniform_(-k_p,k_p)
        self.grad_weight = None
        self.grad_bias = None
    
    def forward(self , *input, in_feature, out_feature):
        if(isinstance(input, tuple)):
            liste = []
            t = input[0]
            for x in t:
                liste.append(x.matmul(weight)+bias) 
            output = tuple(liste)
        else:
            output = input.matmul(weight)+bias  
        
    def backward(self , *gradwrtoutput):
        derivative = []
        gradaccumulated = []
        for x in self.input:
            #derivative.append(1 - torch.tanh(x).pow(2))

        for i in range (len(derivative)):
            #gradaccumulated.append(derivative[i]*gradwrtoutput[i])
        output = tuple(gradaccumulated)
        return output 
        
    def param(self):
        output = [self.weight, self.grad_weight, self.bias, self.grad_bias]
        return output
    
    #https://pytorch.org/docs/stable/nn.html#linear-layers

In [39]:
x1 = torch.empty(3,4,7,10).normal_()
print(x1)
in_feature = 10
out_feature = 2
k = 1/in_feature
k_p = math.sqrt(k)
y = torch.empty(in_feature,out_feature).uniform_(-k_p,k_p)
print(y)
bias = torch.empty(out_feature).uniform_(-k_p,k_p)
print(bias)
print(bias.size())
z1 = x1.matmul(y) + bias
print(z1.size())

tensor([[-0.0678, -0.0091],
        [ 0.3138,  0.0206],
        [ 0.0406,  0.1020],
        [-0.0521, -0.2344],
        [ 0.0756, -0.2297],
        [ 0.3056,  0.0490],
        [-0.0034,  0.0890],
        [ 0.0847,  0.1470],
        [ 0.2353, -0.0367],
        [ 0.3008, -0.2754]])
tensor([ 0.2924, -0.2238])
torch.Size([2])
torch.Size([3, 4, 7, 2])


# RELU (problème dérivée)

In [39]:
#RELU MODULE
class ReLU():
    def __init__(self):
        self.input = None
    
    def forward(self, *input): 
        liste = []
        self.input = input
        for x in input:
            liste.append(torch.max(x,torch.empty(x.size(0), x.size(1)).fill_(0))) 
        output = tuple(liste)
        return output
        
    def backward(self, *gradwrtoutput):
        derivative = []
        gradaccumulated = []
        for x in self.input:
            derivative.append((x.sign()+1)/2) 
            #Si >0, sign donne 1, +1 /2 donc 1 (d(x)/dx = 1)
            #Si =0, sign donne 0, +1 /2 donc 0.5 => Normalement on a pas de cas qui vaut 0, mais à voir
            #Si <0, sign donne -1, +1 /2 donc 0 (d(0)/dx = 0) 
        for i in range (len(derivative)):
            gradaccumulated.append(derivative[i]*gradwrtoutput[i])
        output = tuple(gradaccumulated)
        return output   

    def param(self): 
        return []
    
#backward should get as input a tensor or a tuple of tensors containing the gradient of the 
#loss with respect to the module’s output, accumulate the gradient wrt the parameters, 
#and return a tensor or a tuple of tensors containing the gradient of the loss wrt the module’s input.

In [40]:
T = ReLU()
x = torch.empty(1,2).fill_(3)
y = torch.empty(1,2).fill_(-2)
grad1 = torch.empty(1,2).fill_(2)
grad2 = torch.empty(1,2).fill_(2)
print(x)
print(y)
print(grad1)
print(grad2)
t = T.forward(x, y)
print('ReLU of x, y')
print(t)
grad = T.backward(grad1,grad2)
print('grad')
print(grad)

tensor([[3., 3.]])
tensor([[-2., -2.]])
tensor([[2., 2.]])
tensor([[2., 2.]])
ReLU of x, y
(tensor([[3., 3.]]), tensor([[0., 0.]]))
grad
(tensor([[2., 2.]]), tensor([[0., 0.]]))


# TANH

In [35]:
#TANH MODULE
class Tanh():
    def __init__(self):
        self.input = None
        
    def forward(self, *input): 
        liste = []
        self.input = input
        for x in input:
            liste.append(torch.tanh(x)) 
        output = tuple(liste)
        return output
    
    def backward(self, *gradwrtoutput):
        derivative = []
        gradaccumulated = []
        for x in self.input:
            derivative.append(1 - torch.tanh(x).pow(2))

        for i in range (len(derivative)):
            gradaccumulated.append(derivative[i]*gradwrtoutput[i])
        output = tuple(gradaccumulated)
        return output   

    def param(self):
        return [] #Pas de param

In [36]:
T = Tanh()
x = torch.empty(1,2).fill_(1)
y = torch.empty(1,2).fill_(2)
grad1 = torch.empty(1,2).fill_(2)
grad2 = torch.empty(1,2).fill_(2)
print(x)
print(y)
print(grad1)
print(grad2)
t = T.forward(x, y)
print('tanh of x, y')
print(t)
grad = T.backward(grad1,grad2)
print('grad')
print(grad)

tensor([[1., 1.]])
tensor([[2., 2.]])
tensor([[2., 2.]])
tensor([[2., 2.]])
tanh of x, y
(tensor([[0.7616, 0.7616]]), tensor([[0.9640, 0.9640]]))
grad
(tensor([[0.8399, 0.8399]]), tensor([[0.1413, 0.1413]]))


# LOSSMSE (normalement juste, toujours à la fin)

In [37]:
#LOSSMSE MODULE
#Maybe: convert to list/tensor the tuple
    
class LossMSE():
    def __init__(self):
        self.loss = None
        self.gradloss = None
    
    def forward(self, *input, target): #Si plusieurs input et target comment on fait?
        liste = []
        for i in range (len(input)):
            liste.append((input[i]-target[i]).pow(2)) 
        output = tuple(liste)
        self.loss = output
        return output
        
    def backward(self):
        gradaccumulated = []
        for x in self.loss:
            gradaccumulated.append(2*((x).pow(1/2))) #on obtient 2*(sqrt((x-target).pow(2))) du coup 2*(x-target)
        output = tuple(gradaccumulated)      
        self.gradloss = output
        return output

    def param(self):
        return [] #No param

In [38]:
N = LossMSE()
x = torch.empty(1,2).fill_(1)
y = torch.empty(1,2).fill_(2)
print(x)
print(y)
t = torch.empty(1,2).fill_(-1)
t2 = torch.empty(1,2).fill_(-1)
print(t)
print(t2)
l = N.forward(x, y , target= [t, t2])
print('loss')
print(l)
dl = N.backward()
print('dloss')
print(dl)

tensor([[1., 1.]])
tensor([[2., 2.]])
tensor([[-1., -1.]])
tensor([[-1., -1.]])
loss
(tensor([[4., 4.]]), tensor([[9., 9.]]))
dloss
(tensor([[4., 4.]]), tensor([[6., 6.]]))


# SEQUENTIAL

In [ ]:
#SEQUENTIAL MODULE (to combine several modules in basic sequential structure)
class Sequential(object):
    def forward(self , *input):
        raise NotImplementedError
    def backward(self , *gradwrtoutput):
        raise NotImplementedError
    def param(self): 
        return []
    
#forward should get for input, and returns, a tensor or a tuple of tensors.

#backward should get as input a tensor or a tuple of tensors containing the gradient of the 
#loss with respect to the module’s output, accumulate the gradient wrt the parameters, 
#and return a tensor or a tuple of tensors containing the gradient of the loss wrt the module’s input.

#param should return a list of pairs, each composed of a parameter tensor, and a gradient tensor of same size. 
#This list should be empty for parameterless modules (e.g. ReLU).
#Some modules may requires additional methods, and some modules may keep track of information from the forward 
#pass to be used in the backward.

# Test.py

In [34]:
training_set = torch.empty(1000,2).uniform_(0,1)  #x et y
training_classes = torch.empty(1000,1)
testing_set = torch.empty(1000,2).uniform_(0,1)  #x et y
testing_classes = torch.empty(1000,1)

r = torch.empty(1,1).fill_(1/(2*math.pi)).pow(1/2)

for i in range (1000):
    if (training_set[i].pow(2).sum()).pow(1/2).item() < r.item():
        training_classes[i] = 1
    else:
        training_classes[i] = 0
    
    if (testing_set[i].pow(2).sum()).pow(1/2).item() < r.item():
        testing_classes[i] = 1
    else:
        testing_classes[i] = 0

#builds a network with two input units, two output units, three hidden layers of 25 units,
#trains it with MSE, logging the loss,
#computes and prints the ﬁnal train and the test errors.